** A few sub-tables / queries potentially usefull for SAPS II computation **

* Load librairies / Access the DB

In [ ]:
import numpy as np
omop_login = np.load('full_omop_login.npy').item()

from fleming_lib.tools import connect_to_omop
conn = connect_to_omop(login_dict=omop_login)

import pandas as pd

* Example 

In [ ]:
query = """
SELECT * 
FROM person
LIMIT 10;
"""
pd.read_sql_query(query, conn)

* ICU STAYS : Number

In [ ]:
query = """
SELECT COUNT(distinct visit_detail_id) AS num_totalstays_count
FROM visit_detail
WHERE visit_detail_concept_id = 581382                             -- concept.concept_name = 'Inpatient Intensive Care Facility'
AND visit_type_concept_id = 2000000006;                            -- concept.concept_name = 'Ward and physical location'
"""
pd.read_sql_query(query, conn)

* ICU STAYS : Components

In [1]:
query_icu = """
SELECT *
FROM visit_detail
WHERE visit_detail_concept_id = 581382              
AND visit_type_concept_id = 2000000006
LIMIT 10; 
"""
pd.read_sql_query(query_icu, conn)

NameError: name 'pd' is not defined

* ICU STAYS : Encapsulation

In [ ]:
query_icu = """
WITH ie AS                                                 -- Equivalent to icustays
( SELECT *
  FROM visit_detail
  WHERE visit_detail_concept_id = 581382              
  AND visit_type_concept_id = 2000000006 
)

SELECT *
FROM ie
LIMIT 10;
"""
pd.read_sql_query(query_icu, conn)

* ICU STAYS : Full score computation (so far... DOES NOT WORK)

In [ ]:
query = """
WITH ie AS                                                 -- Equivalent to icustays
( SELECT *
  FROM visit_detail
  WHERE visit_detail_concept_id = 581382              
  AND visit_type_concept_id = 2000000006 
  
) , cpap AS           

( SELECT distinct ie.person_id,                     
    , min(measurement_datetime - interval '1' hour) as starttime
    , max(measurement_datetime + interval '4' hour) as endtime
    --, max(case when lower(ce.value) similar to '%(cpap mask|bipap mask)%' then 1 else 0 end) as cpap
    
  FROM ie
  
  INNER JOIN measurement me                           -- chartevents have been mapped onto MEASUREMENT and OBSERVATION on OMOP..
    ON ie.person_id=me.person_id
    AND me.measurement_datetime between ie.visit_start_datetime and ie.visit_start_datetime + interval '1' day 
  
  WHERE measurement_id in (???)
  -- AND lower(ce.value) similar to '%(cpap mask|bipap mask)%'
  group by ie.person_id
  
)

"""

pd.read_sql_query(query, conn)